# 🩻 JointWise: Multi-Task Medical Image Analysis
## Ensemble Learning Pipeline for Knee MRI Classification and Localization

This notebook implements a comprehensive pipeline for:
- **Data Processing**: Converting FastMRI H5 to DICOM and PNG formats
- **Single-Task Learning**: Classification of ACL tears vs Meniscus tears
- **Multi-Task Learning**: Combined classification and bounding box regression
- **Ensemble Methods**: Advanced model combination techniques

### Pipeline Overview:
1. **Data Setup & Preprocessing**
2. **Single-Task Model Training**
3. **Multi-Task Learning Implementation**
4. **Ensemble Learning & Evaluation**
5. **Results Analysis & Visualization**

# 📦 1. Environment Setup & Imports

In [ ]:
# Essential imports for ensemble learning pipeline
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# PyTorch deep learning framework
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
import torchvision
from torchvision import models, transforms
import timm  # For advanced model architectures

# Data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Machine learning utilities
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    precision_recall_curve, roc_curve, accuracy_score
)
from sklearn.utils.class_weight import compute_class_weight

# Genetic algorithm for hyperparameter optimization
import deap
from deap import base, creator, tools, algorithms
import random
from tqdm import tqdm
import time

# Medical data processing
import h5py
import pydicom
from pydicom.dataset import Dataset as DicomDataset, FileMetaDataset
from pydicom.uid import generate_uid
import xmltodict
from PIL import Image
import datetime
import os

print("✅ All imports successful!")

In [ ]:
# Environment configuration
# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
random.seed(42)

# Set deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Environment setup complete.")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print(f"Using device: {device}")

# 🔄 2. Data Processing Functions

In [ ]:
# FastMRI to DICOM conversion function
def fastmri_to_dicom(filename: Path,
    reconstruction_name: str,
    outfolder: Path,
    flip_up_down: bool = False,
    flip_left_right: bool = False) -> None:
    """
    Convert FastMRI H5 files to DICOM format
    
    Args:
        filename: Path to H5 file
        reconstruction_name: Name of reconstruction in H5 file
        outfolder: Output directory for DICOM files
        flip_up_down: Whether to flip image vertically
        flip_left_right: Whether to flip image horizontally
    """
    fileparts = os.path.splitext(filename.name)
    patientName = fileparts[0]
    f = h5py.File(filename,'r')
    
    if not outfolder:
        outfolder = Path(patientName)
        outfolder.mkdir(parents=True, exist_ok=True)
    
    if 'ismrmrd_header' not in f.keys():
        raise Exception('ISMRMRD header not found in file')
    if reconstruction_name not in f.keys():
        raise Exception('Reconstruction name not found in file')
    
    # Parse header information
    head = xmltodict.parse(f['ismrmrd_header'][()])
    reconSpace = head['ismrmrdHeader']['encoding']['reconSpace']
    measurementInformation = head['ismrmrdHeader']['measurementInformation']
    acquisitionSystemInformation = head['ismrmrdHeader']['acquisitionSystemInformation']
    H1resonanceFrequency_Hz = head['ismrmrdHeader']['experimentalConditions']['H1resonanceFrequency_Hz']
    sequenceParameters = head['ismrmrdHeader']['sequenceParameters']
    
    # Calculate pixel spacing
    pixelSizeX = float(reconSpace['fieldOfView_mm']['x'])/float(reconSpace['matrixSize']['x'])
    pixelSizeY = float(reconSpace['fieldOfView_mm']['y'])/float(reconSpace['matrixSize']['y'])
    
    # Load and process image data
    img_data = f[reconstruction_name][:]
    slices = img_data.shape[0]
    
    if flip_left_right:
        img_data = img_data[:, :, ::-1]
    if flip_up_down:
        img_data = img_data[:, ::-1, :]
    
    # Scale image data
    image_max = 1024
    scale = image_max / np.percentile(img_data, 99.9)
    pixels_scaled = np.clip((scale * img_data), 0, image_max).astype('int16')
    
    # Calculate window parameters
    windowWidth = 2 * (np.percentile(pixels_scaled, 99.9) - np.percentile(pixels_scaled, 0.1))
    windowCenter = windowWidth/2
    
    # Generate unique UIDs
    studyInstanceUid = generate_uid('999.')
    seriesInstanceUid = generate_uid('9999.')
    
    # Create DICOM files for each slice
    for s in range(0, slices):
        slice_filename = "%s_%03d.dcm"%(patientName, s)
        slice_full_path = outfolder/slice_filename
        slice_pixels = pixels_scaled[s,:,:]
        
        # Create DICOM file metadata
        file_meta = FileMetaDataset()
        file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
        file_meta.MediaStorageSOPInstanceUID = generate_uid()
        file_meta.ImplementationClassUID = generate_uid()
        
        # Create main dataset
        ds = DicomDataset()
        ds.file_meta = file_meta
        
        # Set basic DICOM tags
        ds.is_little_endian = True
        ds.is_implicit_VR = True
        ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.4'
        ds.SOPInstanceUID = file_meta.MediaStorageSOPInstanceUID
        ds.StudyInstanceUID = studyInstanceUid
        ds.SeriesInstanceUID = seriesInstanceUid
        ds.InstanceNumber = str(s + 1)
        ds.PatientName = patientName
        ds.PatientID = patientName
        
        # Set image-specific tags
        ds.Modality = 'MR'
        ds.StudyDate = datetime.datetime.now().strftime('%Y%m%d')
        ds.StudyTime = datetime.datetime.now().strftime('%H%M%S')
        ds.SeriesDate = ds.StudyDate
        ds.SeriesTime = ds.StudyTime
        
        # Image data
        ds.SamplesPerPixel = 1
        ds.PhotometricInterpretation = "MONOCHROME2"
        ds.PixelRepresentation = 1
        ds.HighBit = 15
        ds.BitsStored = 16
        ds.BitsAllocated = 16
        ds.Columns = slice_pixels.shape[1]
        ds.Rows = slice_pixels.shape[0]
        ds.PixelSpacing = [pixelSizeY, pixelSizeX]
        ds.WindowCenter = str(int(windowCenter))
        ds.WindowWidth = str(int(windowWidth))
        ds.PixelData = slice_pixels.tobytes()
        
        # Save DICOM file
        ds.save_as(slice_full_path)
    
    f.close()
    print(f"Converted {slices} slices to DICOM format in {outfolder}")

In [ ]:
# DICOM to PNG conversion function
def dicom_to_png(dicom_path, output_path, window_center=None, window_width=None):
    """
    Convert DICOM file to PNG format with proper windowing
    
    Args:
        dicom_path: Path to DICOM file
        output_path: Output path for PNG file
        window_center: Window center value (optional)
        window_width: Window width value (optional)
    """
    try:
        # Read DICOM file
        dicom_data = pydicom.dcmread(dicom_path)
        pixel_array = dicom_data.pixel_array
        
        # Apply windowing if specified
        if window_center is None:
            window_center = getattr(dicom_data, 'WindowCenter', None)
        if window_width is None:
            window_width = getattr(dicom_data, 'WindowWidth', None)
        
        if window_center is not None and window_width is not None:
            if isinstance(window_center, (list, tuple)):
                window_center = window_center[0]
            if isinstance(window_width, (list, tuple)):
                window_width = window_width[0]
            
            # Apply windowing
            min_val = window_center - window_width // 2
            max_val = window_center + window_width // 2
            pixel_array = np.clip(pixel_array, min_val, max_val)
        
        # Normalize to 0-255 range
        pixel_array = pixel_array.astype(np.float64)
        pixel_array = (pixel_array - pixel_array.min()) / (pixel_array.max() - pixel_array.min())
        pixel_array = (pixel_array * 255).astype(np.uint8)
        
        # Save as PNG
        image = Image.fromarray(pixel_array)
        image.save(output_path)
        
        return True
    except Exception as e:
        print(f"Error converting {dicom_path}: {e}")
        return False

# 📁 3. Data Setup & Loading

In [ ]:
# Data paths configuration
# Update these paths according to your data structure
DATA_ROOT = Path("/home/bictor0301/Code/JointWise")
FASTMRI_PATH = DATA_ROOT / "fastmri_data"  # Adjust to your FastMRI data location
DICOM_OUTPUT = DATA_ROOT / "output-folder"
PNG_OUTPUT = DATA_ROOT / "png-output"
ANNOTATIONS_PATH = DATA_ROOT / "annotations"

# Create output directories
DICOM_OUTPUT.mkdir(exist_ok=True)
PNG_OUTPUT.mkdir(exist_ok=True)

print(f"Data root: {DATA_ROOT}")
print(f"DICOM output: {DICOM_OUTPUT}")
print(f"PNG output: {PNG_OUTPUT}")
print(f"Annotations: {ANNOTATIONS_PATH}")

In [ ]:
# Load and process annotations
def load_annotations():
    """
    Load annotation files and create unified dataset
    """
    try:
        # Try to load annotation files
        annotation_files = list(ANNOTATIONS_PATH.glob("*.csv"))
        
        if not annotation_files:
            print("⚠️ No annotation files found. Creating sample data for demonstration.")
            return create_sample_data()
        
        # Load annotations
        annotations = pd.concat([pd.read_csv(f) for f in annotation_files], ignore_index=True)
        print(f"✅ Loaded {len(annotations)} annotations from {len(annotation_files)} files")
        return annotations
        
    except Exception as e:
        print(f"❌ Error loading annotations: {e}")
        print("Creating sample data for demonstration.")
        return create_sample_data()

def create_sample_data():
    """
    Create sample dataset for demonstration if real data is not available
    """
    np.random.seed(42)
    n_samples = 1000
    
    # Create sample annotations
    sample_data = {
        'file_id': [f'sample_{i:04d}' for i in range(n_samples)],
        'slice': np.random.randint(0, 20, n_samples),
        'label': np.random.choice(['ACL_tear', 'Meniscus_tear'], n_samples),
        'bbox_x': np.random.randint(50, 200, n_samples),
        'bbox_y': np.random.randint(50, 200, n_samples),
        'bbox_width': np.random.randint(50, 100, n_samples),
        'bbox_height': np.random.randint(50, 100, n_samples),
    }
    
    annotations = pd.DataFrame(sample_data)
    print(f"✅ Created {len(annotations)} sample annotations for demonstration")
    return annotations

# Load the annotations
annotations = load_annotations()
print(f"Dataset shape: {annotations.shape}")
print(f"Columns: {list(annotations.columns)}")
if len(annotations) > 0:
    print(f"
Label distribution:")
    print(annotations['label'].value_counts())